In [1]:
import os
os.chdir("..")   # go from notebooks/ to project root

In [2]:
from __future__ import annotations

from flamekit.datasets import extract_full_field_csv
from flamekit.io_fronts import Case
from mpi4py import MPI
from pathlib import Path

# -----------------------------
# Find project root robustly
# -----------------------------

def find_project_root(start: Path | None = None) -> Path:
    p = (start or Path.cwd()).resolve()
    for cand in [p, *p.parents]:
        if (cand / "flamekit").is_dir():
            return cand
    raise RuntimeError("Could not find project root (folder containing 'flamekit').")


PROJECT_ROOT = find_project_root()

# INPUT PARAMETERS (CAPITALS)
# ============================================================

TIME_STEP = 335
MULTI_TIME_STEP = True

TIME_STEP_START = 270
TIME_STEP_END = 301

PHI = 0.40
LAT_SIZE = "025"
T_REF = 300
P_REF = 1e05

POST = True

DATA_BASE_DIR = PROJECT_ROOT / "data"
OUTPUT_BASE_DIR = PROJECT_ROOT / "isocontours"  # keep using your unified folder convention

FILE_NAME = "po_postPremix" if POST else "premix"

SCALARS = [
    "H2", "O2", "H2O", "H", "O", "OH", "HO2", "H2O2", "N2", "HRR",
    "curvature", "stretch_rate", "DW_FDS",
    "abs_flame_prop_vel_normal",
    "flow_velocity_normal", "flow_velocity_tangential",
    "strain_rate", "tangential_strain_rate", "normal_strain_rate",
    "density_ratio_sigma", "gradT", "gradT_normal",
    "total_heat_conduction", "heat_conduction_normal", "heat_conduction_tangential",
    "FDS_src_term", "FDS_diffusion_term", "FDS_diff_velocity_term",
    "H2_diffusion_total", "O2_diffusion_total", "H_diffusion_total",
    "vorticity",
]

CANTERA_YAML = PROJECT_ROOT / "flamekit" / "chemical_mech" / "BurkeH2.yaml"
CANTERA_INPUTS = [str(CANTERA_YAML), None, T_REF, P_REF]

# RUN
# ============================================================

comm = MPI.COMM_WORLD
rank = comm.rank


def _process_one_timestep(ts: int) -> None:
    case = Case(
        base_dir=OUTPUT_BASE_DIR,
        phi=PHI,
        lat_size=LAT_SIZE,
        time_step=ts,
        post=POST,
    )

    out_path = extract_full_field_csv(
        case,
        data_base_dir=DATA_BASE_DIR,
        output_base_dir=OUTPUT_BASE_DIR,
        file_name=FILE_NAME,
        scalar_names=SCALARS,
        comm=comm,
        compute_vel_jacobian=False,
        compute_vel_hessian=False,
        compute_reaction_rates=True,
        compute_progress_var=True,
        cantera_inputs=CANTERA_INPUTS,
    )

    if rank == 0:
        print(f"[rank0] Saved full field: {out_path}")


if MULTI_TIME_STEP:
    for ts in range(TIME_STEP_START, TIME_STEP_END + 1):
        if rank == 0:
            print(f"=== Processing TIME_STEP={ts} ===")
        _process_one_timestep(ts)
else:
    if rank == 0:
        print(f"=== Processing TIME_STEP={TIME_STEP} ===")
    _process_one_timestep(TIME_STEP)



=== Processing TIME_STEP=270 ===
2026-01-01 17:08:22,420 - Mesh - INFO - Initializing empty Mesh object.
2026-01-01 17:08:22,420 - Field - INFO - Initializing empty Field object
2026-01-01 17:08:22,421 - pynekread - INFO - Reading file: /media/alexandros/OS/Users/alexp/Documents/Bachelor Thesis/Code/data/phi0.40/h400x025_ref/po_postPremix0.f00001
2026-01-01 17:08:22,454 - Mesh - INFO - Initializing Mesh object from x,y,z ndarrays.
2026-01-01 17:08:22,454 - Mesh - INFO - Initializing common attributes.
2026-01-01 17:08:22,455 - Mesh - INFO - Getting vertices
2026-01-01 17:08:22,457 - Mesh - INFO - Getting edge centers
2026-01-01 17:08:22,483 - Mesh - INFO - Facet centers not available for 2D
2026-01-01 17:08:22,484 - Mesh - INFO - Mesh object initialized.
2026-01-01 17:08:22,484 - Mesh - INFO - Mesh data is of type: float64
2026-01-01 17:08:22,485 - Mesh - INFO - Elapsed time: 0.030748648999999996s
2026-01-01 17:08:22,485 - pynekread - INFO - Reading field data
2026-01-01 17:08:22,485 -

KeyboardInterrupt: 